In [1]:
%run graphical_functions.ipynb

### Preparing functions to load and save distances

In [ ]:
def load_distance_cache():
    """
    Loads the distance cache from a JSON file into a dictionary. Converts string keys
    back to tuple keys if necessary.

    Returns:
        dict: A dictionary with keys as tuples representing city pairs and values as the
        distances between these cities. Returns an empty dictionary if the file is not
        found or if there is a decoding error.
    """
    try:
        with open("distance_cache.json", "r") as file:
            loaded_cache = json.load(file)
            # Convert string keys back to tuples if your logic requires tuple keys
            return {tuple(key.split(",")): value for key, value in loaded_cache.items()}
    except (FileNotFoundError, json.JSONDecodeError):
        return {}  # Return an empty dictionary if there is no file or decoding fails


def save_distance_cache(cache):
    """
    Saves the given cache dictionary to a JSON file. Converts tuple keys in the dictionary
    to a string format suitable for JSON storage.

    Args:
        cache (dict): The dictionary with tuple keys and distance values to save.
    """
    with open("distance_cache.json", "w") as file:
        # Convert tuple keys to a string format
        formatted_cache = {",".join(key): value for key, value in cache.items()}
        json.dump(formatted_cache, file, indent=4)

### Retrieving the cities names and coordinates from a dataset

In [ ]:
def read_cities_csv_to_dict(csv_file_path):
    """
    Reads a CSV file containing city information and returns a dictionary
    with city names as keys and their coordinates (latitude and longitude) as values.

    Args:
    csv_file_path: The file path to the CSV file containing the city data.

    Returns:
    A dictionary with city names as keys and (latitude, longitude) tuples as values.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path, delimiter=",")

    duplicates = df[df.duplicated(keep=False)]
    if duplicates.empty:
        print("No duplicates found.")
    else:
        print("Duplicates found:", len(duplicates))
        df.drop_duplicates(keep="first", inplace=True)

    # Create a dictionary from the DataFrame
    city_coordinates = {
        row["city"]: (row["lat"], row["lng"])
        for index, row in df.iterrows()
        if row["country"] == "Sweden"
    }
    print(city_coordinates)

    return city_coordinates

### Initiating needed values

In [2]:
current_file_directory = os.path.dirname(os.path.abspath(__file__))
dotenv_path = os.path.join(current_file_directory, '.env')
# Load environment variables from .env file
load_dotenv(dotenv_path)

# Access the environment variable
my_api_key = os.getenv("GOOGLE_KEY")

gmaps = googlemaps.Client(key=my_api_key)

# Get the distances file as soon as we load
distance_cache = load_distance_cache()

csv_file_path = "./datasets/Sweden_cities.csv"
cities_coordinates = read_cities_csv_to_dict(csv_file_path)

### Function to get the distances between the cities of a specific instance

In [ ]:
def fetch_distances(selected_cities):
    """
    Fetches the driving distances between each pair of selected cities using Google Maps API.
    Updates and uses a local cache to minimize API calls.

    Args:
        selected_cities (list): A list of selected city names.

    Returns:
        dict: A nested dictionary where each key is a city name from the selected cities and each value is another dictionary.
            The nested dictionary's keys are the other cities in the list, and the values are the distances to these cities in meters.
    """
    print("started fetch_distances")
    distances = {}
    cache_updated = False
    for origin in selected_cities:
        distances[origin] = {}
        for destination in selected_cities:
            if origin == destination:
                distances[origin][destination] = 0
            else:
                # Sort and convert to string to use as a JSON-compatible key
                cache_key = tuple(sorted([origin, destination]))
                str_cache_key = ",".join(cache_key)
                if cache_key in distance_cache:
                    distances[origin][destination] = distance_cache[cache_key]
                    print("Found cache")
                else:
                    try:
                        print("started API call")
                        result = gmaps.distance_matrix(
                            origins=[cities_coordinates[origin]],
                            destinations=[cities_coordinates[destination]],
                            mode="driving",
                        )
                        distance = result["rows"][0]["elements"][0]["distance"]["value"]
                        distances[origin][destination] = distance
                        distance_cache[cache_key] = distance  # Store using tuple
                        cache_updated = True
                        print("finished API call")
                    except Exception as e:
                        print(
                            f"Error fetching distance between {origin} and {destination}: {e}"
                        )
                        distances[origin][destination] = float(
                            "inf"
                        )  # Assign a high cost in case of error
    if cache_updated:
        save_distance_cache(distance_cache)
    print("finished fetch_distances")
    return distances

### Function to get the cost of any solution

In [ ]:
def calculate_cost(solution, distances):
    """
    Calculates the total travel cost of a given solution based on the distances between cities.

    Args:
        solution (list): An ordered list of city names representing the tour.
        distances (dict): A nested dictionary of distances between each pair of cities.

    Returns:
        float: The total cost of the solution in kilometers.
    """
    # Calculate the total distance in meters first
    total_distance_meters = (
        sum(distances[solution[i]][solution[i + 1]] for i in range(len(solution) - 1))
        + distances[solution[-1]][solution[0]]
    )
    # Convert the total distance to kilometers
    total_distance_km = total_distance_meters / 1000.0
    return total_distance_km